***Welcome to Medical image processing in Python***<br/>

Presented by: Reza Saadatyar (2024-2025) <br/>
E-mail: Reza.Saadatyar@outlook.com 

**Import the require library**

In [1]:
import sys
import numpy as np
from tqdm import tqdm
from PIL import Image
from Functions import Data_path
import matplotlib.pyplot as plt
from skimage import io
from colorama import Back, Fore
# from skimage.viewer import ImageViewer
# from OOP import Pre_Processing_R2g
# from Functions import image_resizer

**1. Set Image Path & Load data**

In [8]:
folder_path = "D:/Medical-Image-Processing/Data/Inputs/"
files_inputs, _, _ = Data_path.data_path(folder_path, data_format="tif")

folder_path = "D:/Medical-Image-Processing/Data/Masks/"
files_masks, _, _ = Data_path.data_path(folder_path, data_format="TIF")

*Image-Width: n*<br/>
*Image-Height: m*<br/>
*Channels: c*<br/>
*Planes: p*<br/>
*Grayscale: (p, m, n)*<br/>
*RGB: (p, m, n, c)*<br/>

**2. Convert the image into an array**

In [ ]:
import numpy as np  # Import numpy for array manipulation
from skimage import io  # Import scikit-image library for image I/O operations

class ImageDatasetLoader:
    """
    A class to load and process images from file paths into a NumPy array.
    """
    def __init__(self, file_paths: str) -> None:
        """
        Initialize the ImageDatasetLoader class.
        :param file_paths: List of file paths to images.
        """
        self.images = None  # Placeholder for loaded image array
        self.labels = None  # Placeholder for labels, if any
        self.img_width = None  # Width of the images
        self.img_height = None  # Height of the images
        self.img_channels = None  # Number of color channels in the images
        self.file_paths = file_paths  # Store the file input list

    @property
    def load_images(self) -> np.ndarray:
        """
        Load images from the provided file paths into a NumPy array.
        """
        if not self.file_paths:
            raise ValueError("No file paths provided.")  # Raise an error if no files are given

        num_files = len(self.file_paths)  # Number of input files
        data = io.imread(self.file_paths[0])  # Load the first image to get dimensions

        if data.ndim == 2:  # Check if the image is grayscale
            self.img_height, self.img_width = data.shape  # Get dimensions for grayscale image
            self.images = np.zeros(  # Initialize array for grayscale images
                (num_files, self.img_height, self.img_width),
                dtype=np.uint8  # Use unsigned 8-bit integer for pixel values
            )
        else:  # Image is colored (e.g., RGB)
            self.img_height, self.img_width, self.img_channels = data.shape  # Get dimensions for colored image
            self.images = np.zeros(  # Initialize array for colored images
                (num_files, self.img_height, self.img_width, self.img_channels),
                dtype=np.uint8
            )

        for idx, file_path in enumerate(self.file_paths):  # Iterate through all file paths
            self.images[idx] = io.imread(file_path)  # Read each image and store in the array

        return self.images  # Return the loaded image array

**2.1. Convert the images in the "Inputs" folder to a NumPy array**

In [ ]:
inputs = ImageDatasetLoader(files_inputs) # A class to load and process images from file paths into a NumPy array.
inputs.load_images.shape  # Load images from the provided file paths into a NumPy array.

(58, 768, 896, 3)

In [ ]:
masks = ImageDatasetLoader(files_masks) # A class to load and process images from file paths into a NumPy array.
masks.load_images.shape  # Load images from the provided file paths into a NumPy array.

(58, 768, 896)

**2.2. Convert the images in the "Masks" folder to boolean**

In [49]:
data = io.imread(files_inputs[0])  # Load a file for obtaining size data
img_height = data.shape[0]         # Get the height of the image
img_width = data.shape[1]          # Get the width of the image
img_channels = data.shape[2]       # Get the number of channels in the image
labels = np.zeros((len(files_inputs), img_height, img_width, 2), dtype = bool)  # Shape: [num_files, H, W, 2]

sys.stdout.flush()
for ind, _ in tqdm(enumerate(files_inputs)):  # Progressively iterate through all the input files
    mask = np.squeeze(io.imread(files_masks[ind])).astype(bool)  # Load and convert each mask to boolean
    labels[ind, :, :, 0] = ~mask  # Background (inverse of mask)
    labels[ind, :, :, 1] = mask   # Foreground (actual mask)

0it [00:00, ?it/s]

58it [00:00, 92.12it/s] 


**[Image Resizing](https://scikit-image.org/docs/stable/api/skimage.transform.html#skimage.transform.resize)**<br/>
`Standardizing Image Dimensions:` Machine learning models, like CNNs, need input data with fixed dimensions. For instance, if a model requires images of size 224x224x3, all input images must be resized to that shape.<br/>
`Reducing Computational Load:`Resizing images to smaller dimensions lowers computational costs, particularly with large datasets, and aids in faster training or inference for deep learning models.

**Augmentation, Re_Color, & Im_Saving**<br/>

In [ ]:
import os
import glob
import shutil
import numpy as np
from skimage import transform, color  
from tensorflow import keras

#  ================================= Class for resizing & converting images ====================================
class Image:
    def __init__(self, img_data: np.ndarray):
        """
        :param imgs: A numpy array of shape [num_images, height, width, channels].
        """
        self.img_data = img_data
        
    # ---------------------------------------------- Resizes ---------------------------------------------------
    def resize_images(self, img_height_resized: int, img_width_resized: int, img_channels: int) -> np.ndarray:
        """
        Resizes a batch of images to the specified height, width, and channels.

        
        :return: A numpy array of resized images.
        """
        # Initialize an empty array to store resized images
        resized_imgs = np.zeros(
            (self.img_data.shape[0], img_height_resized, img_width_resized, img_channels),
            dtype=np.uint8
        )
        
        # Loop through each image in the batch
        for i in range(self.img_data.shape[0]):
            # Resize the image to the target dimensions and store it
            resized_imgs[i] = transform.resize(
                self.img_data[i],
                (img_height_resized, img_width_resized, img_channels),
                preserve_range=True  # Preserve the range of pixel values
            )
            
        return resized_imgs  # Return the resized images
    
    # ------------------------------------------ RGB images to grayscale ---------------------------------------
    def rgb2gray_scale(self) -> np.ndarray:
        """
        Converts the RGB images to grayscale.

        :return: A numpy array of grayscale images.
        """

        img_num, img_height, img_width, _ = self.img_data.shape
        img_gray = np.zeros((img_num, img_height, img_width), dtype=np.uint8)

        for i in range(img_num):
            # img_gray[i] = (color.rgb2gray(self.img_data[i]) * 255).astype(np.uint8)  # scale back to [0, 255]

            img_gray[i] = color.rgb2gray(self.img_data[i])
        
        return img_gray


# ================================== Class for augmentation (rotation) =========================================
class Augmentation:
    def __init__(self, num_augmented_imag: int, imag_files_path: str, imag_augmented_path: str):
        """
        Applies image augmentation (rotation) to images in the specified directory and saves them.

        :param num_augmented_images: Number of augmented images to generate.
        :param imag_files_path: Path to the directory containing the images.
        :param imag_augmented_path: Path to the directory to save augmented images.
        """
        
        self.imag_files_path = imag_files_path
        self.num_augmented_imag = num_augmented_imag
        self.imag_augmented_path = imag_augmented_path
        
    def augmented_images(self) -> None:
        
        dat = io.imread(glob.glob(self.imag_files_path + '/*')[0])
        self.imag_augmented_path = os.path.join(self.imag_augmented_path, 'Rotated/')
        os.makedirs(self.imag_augmented_path, exist_ok=True)  # Ensure the temporary folder is created
        # Create a temporary folder inside the original folder for processing
        TEMP_DIR = os.path.join(self.imag_files_path, 'Temp/')
        os.makedirs(TEMP_DIR, exist_ok=True)  # Ensure the temporary folder is created
        
        # Copy all files from the main folder to the temporary folder
        for filename in os.listdir(self.imag_files_path):
            if filename.casefold().endswith(('.tif', '.jpg', '.png')):  # Correct usage with a tuple
                shutil.copy(os.path.join(self.imag_files_path, filename), os.path.join(TEMP_DIR, filename))
   
        # Set up the ImageDataGenerator for image augmentation
        Data_Gen = keras.preprocessing.image.ImageDataGenerator(rotation_range=30)  # Rotate images randomly up to 30 degrees
        # Use flow_from_directory to process the images in the Temp folder
        img_aug = Data_Gen.flow_from_directory(
            self.imag_files_path,     # Parent directory of Temp
            classes=['Temp'],    # Specify the subfolder 'Temp' as the target
            batch_size=1,        # Process one image at a time
            save_to_dir=self.imag_augmented_path,  # Save augmented images to the Rotated folder
            save_prefix='Aug',   # Prefix for augmented images
            target_size=(dat.shape[0], dat.shape[1]),  # Resize images to the specified dimensions
            class_mode=None      # No labels, as we're working with unclassified images
        )
        
        for _ in range(self.num_augmented_imag):  # Generate augmented images and save them
            next(img_aug)  # Process the next image and save it

        shutil.rmtree(TEMP_DIR)  # Delete the temporary folder and its contents after processing

    # ---------------------------------------------- Plot ------------------------------------------------------
    def plot_img_original_augment(self, num_img: int) -> None:
        
        _, axs = plt.subplots(nrows=2, ncols=num_img)
        
        # Check if num_img is 1 (special case for 1 image)
        if num_img == 1:
            # Display images on the first row
            # io.imread(files_inputs[0])[:, :, :3].shape
            axs[0].imshow(io.imread(glob.glob(self.imag_files_path + '/*')[0]), cmap='gray')
            axs[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
            [spine.set_visible(False) for spine in axs[0].spines.values()]  # Hide all spines
            axs[0].set_ylabel("Original Images", fontsize=12, labelpad=10)  # Y-axis label for the first row
            
            # Display images on the second row
            axs[1].imshow(io.imread(glob.glob(self.imag_augmented_path + '/*')[0]), cmap='gray')
            axs[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
            [spine.set_visible(False) for spine in axs[1].spines.values()]  # Hide all spines
            axs[1].set_ylabel("Augmented Images", fontsize=12, labelpad=10)  # Y-axis label for the second row

        else:
            for i in range(num_img):
                # Display images on the first row
                axs[0, i].imshow(io.imread(glob.glob(self.imag_files_path + '/*')[i]), cmap='gray')
                axs[0, i].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
                [spine.set_visible(False) for spine in axs[0, i].spines.values()]  # Hide all spines

                # Display images on the second row
                axs[1, i].imshow(io.imread(glob.glob(self.imag_augmented_path + '/*')[i]), cmap='gray')
                axs[1, i].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)  # Hide ticks
                [spine.set_visible(False) for spine in axs[1, i].spines.values()]  # Hide all spines

            # Add ylabel for each row (only set ylabel for the first column of each row)
            axs[0, 0].set_ylabel("Original Images", fontsize=12, labelpad=10)  # Y-axis label for the first row
            axs[1, 0].set_ylabel("Augmented Images", fontsize=12, labelpad=10)  # Y-axis label for the second row

        # Adjust layout to make sure images and titles don't overlap
        plt.tight_layout()

        # Auto-scale to fit the images in the figure area
        plt.autoscale(enable=True, axis='both', tight=True)
        plt.show()


# ==================================== Class for RGB images to grayscale # =====================================
class RGB2Gray:
    def __init__(self, files_path: str) -> None:
        self.files_path = files_path
    
    def rgb2gray_scale(self) -> np.ndarray:
        """
        Converts the RGB images to grayscale.

        :return: A numpy array of grayscale images.
        """
        io.imread(glob.glob(self.files_path + '/*'))
        img_num, img_height, img_width, _ = self.img_data.shape
        img_gray = np.zeros((img_num, img_height, img_width), dtype=np.uint8)

        for i in range(img_num):
            # img_gray[i] = (color.rgb2gray(self.img_data[i]) * 255).astype(np.uint8)  # scale back to [0, 255]

            img_gray[i] = color.rgb2gray(self.img_data[i])
        
        return img_gray


img = Image(inputs)

In [95]:
import os

# Define the folder path
files_path = "D:/Medical-Image-Processing/Data/Inputs"

# Get the full path and filenames of files in the folder (excluding subfolders)
files_with_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) 
                    if os.path.isfile(os.path.join(folder_path, f))]

print(files_with_paths)

['D:/Medical-Image-Processing/Data/Inputs\\ytma12_010804_benign2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma12_010804_benign3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma12_010804_malignant1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma12_010804_malignant2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma12_010804_malignant3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_benign1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_benign2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_benign3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_malignant1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_malignant2_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma23_022103_malignant3_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma49_042003_benign1_ccd.tif', 'D:/Medical-Image-Processing/Data/Inputs\\ytma49_042003_benign2_ccd.tif', 'D:/Medical-Image-P

In [ ]:
# ==================================== Class for RGB images to grayscale # =====================================
class RGB2Gray:
    def __init__(self, files_path: str, save_fig: str="off") -> None:
        self.save_fig = save_fig
        self.files_path = files_path

    @property
    def rgb_convert_gray(self) -> np.ndarray:
        
       # Get the full path and filenames of files in the folder (excluding subfolders)
        files_with_paths = [os.path.join(self.files_path, f) for f in os.listdir(self.files_path) 
                    if os.path.isfile(os.path.join(self.files_path, f))]

        img_num = len(files_with_paths)
        img_height, img_width, _ = io.imread(files_with_paths[0]).shape
        img_gray = np.zeros((img_num, img_height, img_width), dtype=np.uint8)

        for i in range(img_num):
            img_gray[i] = (color.rgb2gray(io.imread(files_with_paths[i])) * 255).astype(np.uint8)  # scale back to [0, 255]
        
        if self.save_fig == "on":
            
            
        return img_gray
    
    # def save_img_gray(self, path_save):
        
    

In [153]:
path_save = 'D:/Medical-Image-Processing/Data/'
img_gray = a 
os.makedirs(os.path.join(path_save, 'Gray image/'), exist_ok=True)  # Ensure the temporary folder is created
for ind, filename in enumerate(os.listdir(files_path)):
     if os.path.isfile(os.path.join(files_path, filename)):
        print(ind, '-->', filename)
      #   io.imsave(fname='{}{}'.format(path_save + 'Gray image/', filename), arr=img_gray[ind])

1 --> ytma12_010804_benign2_ccd.tif
2 --> ytma12_010804_benign3_ccd.tif
3 --> ytma12_010804_malignant1_ccd.tif
4 --> ytma12_010804_malignant2_ccd.tif
5 --> ytma12_010804_malignant3_ccd.tif
6 --> ytma23_022103_benign1_ccd.tif
7 --> ytma23_022103_benign2_ccd.tif
8 --> ytma23_022103_benign3_ccd.tif
9 --> ytma23_022103_malignant1_ccd.tif
10 --> ytma23_022103_malignant2_ccd.tif
11 --> ytma23_022103_malignant3_ccd.tif
12 --> ytma49_042003_benign1_ccd.tif
13 --> ytma49_042003_benign2_ccd.tif
14 --> ytma49_042003_benign3_ccd.tif
15 --> ytma49_042003_malignant1_ccd.tif
16 --> ytma49_042003_malignant2_ccd.tif
17 --> ytma49_042003_malignant3_ccd.tif
18 --> ytma49_042203_benign1_ccd.tif
19 --> ytma49_042203_benign2_ccd.tif
20 --> ytma49_042203_benign3_ccd.tif
21 --> ytma49_042203_malignant1_ccd.tif
22 --> ytma49_042203_malignant2_ccd.tif
23 --> ytma49_042203_malignant3_ccd.tif
24 --> ytma49_042403_benign1_ccd.tif
25 --> ytma49_042403_benign2_ccd.tif
26 --> ytma49_042403_benign3_ccd.tif
27 --> ytma

In [ ]:

New = 'D:/Python/Breast/'

def Pre_Process_Im_Saving(Path_Images, Path_Output, Tensor):
    
    for i, filename in enumerate(os.listdir(Path_Images)):
        
        imsave(fname='{}{}'.format(Path_Output, filename),
               arr=Tensor[i])
        
        print('{}: {}'.format(i, filename))
    
Pre_Process_Im_Saving(IMAGE_PATH, New, Gray_Scale)

In [150]:
rgb_ = RGB2Gray(files_path)
a = rgb_.rgb_convert_gray
a.shape

(51, 768, 896)

In [142]:
a[2][10:15][3]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [22]:
# Resizing the images
resized_images = img.resize_images(img_height_resized=255, img_width_resized=255, img_channels=3)
print(f" Fore.RED + Resizing images from {inputs.shape} to {resized_images.shape}") 

 Fore.RED + Resizing images from (58, 768, 896, 3) to (58, 255, 255, 3)


In [63]:
a = img.rgb2gray_scale()
a.shape

(58, 768, 896)

In [60]:
num_augmented_imag = 3
# Path to the folder containing the original images
imag_files_path = 'D:/Medical-Image-Processing/Data/Inputs/A/B/C/'

# Path where augmented images will be saved
imag_augmented_path = 'D:/Medical-Image-Processing/Data/'
augm = Augmentation(num_augmented_imag, imag_files_path, imag_augmented_path)
augm.augmented_images()
# resizer.augmented_images(num_augmented_imag, imag_files_path, imag_augmented_path)

Found 2 images belonging to 1 classes.


In [ ]:
augm.plot_img_original_augment(num_img=2)

In [20]:
dat = io.imread(glob.glob(imag_files_path + '/*')[0])
dat.shape[0]

768